# Experiments with other libraries #

In [1]:
debug_local = True #to use local version
number_of_folds = 5

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import *
from yspecies.partition import *

In [4]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


## Loading data ##

In [7]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [30]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(414, 12352)",12352,40,414,"(12352, 2)","(41, 19)"


In [36]:
data_cleaned = data.collect(lambda e: e.dropna(axis=1).dropna(axis=0)) #droping NA-s
data_cleaned

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(414, 3896)",3896,40,414,"(12352, 2)","(41, 19)"


In [37]:
from sklearn.pipeline import Pipeline

selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"])


In [38]:
pipe = Pipeline(
    [
        ('extractor', DataExtractor(selection)),
         ("partitioner", DataPartitioner(folds =  5, species_in_validation=2, not_validated_species = ["Homo_sapiens"])), # to partition it according to sorted stratification
    ]
               )

In [39]:
partitions = pipe.fit_transform(data_cleaned)
partitions

<table><tr><th>partitions_X</th><th>partitions_Y</th></tr><tr><td align='left'>[ (93, 3897),(80, 3897),(80, 3897),(83, 3897),(78, 3897) ]</td><td align='left'>[ (93,),(80,),(80,),(83,),(78,) ]</td></tr><tr><th>show(X,10,10)</th><th>show(Y,10,10)</th></tr><tr><td><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>ENSG00000073921</th>
      <th>ENSG00000139687</th>
      <th>ENSG00000135506</th>
      <th>ENSG00000165995</th>
      <th>ENSG00000073756</th>
      <th>ENSG00000097007</th>
      <th>ENSG00000140519</th>
      <th>ENSG00000179915</th>
      <th>ENSG00000068912</th>
      <th>ENSG00000103479</th>
    </tr>
    <tr>
      <th>run</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>SRR5961830</th>
      <td>88.584</td>
      <td>24.830</td>
      <td>37.382</td>
      <td>41.845</td>
      <td>0.195</td>
      <td>9.290</td>
      <td>1.259</td>
      <td>68.349</td>
      <td>57.508</td>
      <td>27.907</td>
    </tr>
    <tr>
      <th>SRR5885325</th>
      <td>156.413</td>
      <td>40.310</td>
      <td>58.443</td>
      <td>52.828</td>
      <td>0.185</td>
      <td>14.129</td>
      <td>2.076</td>
      <td>216.571</td>
      <td>75.357</td>
      <td>39.847</td>
    </tr>
    <tr>
      <th>SRR306843</th>
      <td>46.574</td>
      <td>9.542</td>
      <td>33.752</td>
      <td>16.521</td>
      <td>0.219</td>
      <td>26.296</td>
      <td>0.024</td>
      <td>0.487</td>
      <td>24.300</td>
      <td>17.742</td>
    </tr>
    <tr>
      <th>SRR306844</th>
      <td>174.916</td>
      <td>17.665</td>
      <td>135.480</td>
      <td>2.313</td>
      <td>14.948</td>
      <td>22.953</td>
      <td>0.000</td>
      <td>4.536</td>
      <td>49.815</td>
      <td>49.559</td>
    </tr>
    <tr>
      <th>SRR306845</th>
      <td>254.806</td>
      <td>10.468</td>
      <td>75.646</td>
      <td>2.329</td>
      <td>2.126</td>
      <td>27.842</td>
      <td>27.757</td>
      <td>4.465</td>
      <td>77.774</td>
      <td>40.261</td>
    </tr>
    <tr>
      <th>SRR306841</th>
      <td>79.435</td>
      <td>10.000</td>
      <td>61.993</td>
      <td>31.472</td>
      <td>0.567</td>
      <td>14.133</td>
      <td>0.922</td>
      <td>129.211</td>
      <td>80.421</td>
      <td>16.593</td>
    </tr>
    <tr>
      <th>SRR306840</th>
      <td>84.257</td>
      <td>9.014</td>
      <td>63.076</td>
      <td>30.402</td>
      <td>0.558</td>
      <td>15.244</td>
      <td>0.863</td>
      <td>117.360</td>
      <td>75.213</td>
      <td>16.362</td>
    </tr>
    <tr>
      <th>SRR306839</th>
      <td>82.601</td>
      <td>9.169</td>
      <td>59.656</td>
      <td>29.294</td>
      <td>1.119</td>
      <td>15.953</td>
      <td>2.058</td>
      <td>111.270</td>
      <td>80.849</td>
      <td>16.331</td>
    </tr>
    <tr>
      <th>SRR787277</th>
      <td>183.282</td>
      <td>25.456</td>
      <td>179.877</td>
      <td>3.237</td>
      <td>7.567</td>
      <td>38.311</td>
      <td>0.062</td>
      <td>2.532</td>
      <td>59.197</td>
      <td>68.384</td>
    </tr>
    <tr>
      <th>SRR1521445</th>
      <td>217.140</td>
      <td>8.631</td>
      <td>80.511</td>
      <td>2.106</td>
      <td>0.658</td>
      <td>18.901</td>
      <td>55.892</td>
      <td>8.864</td>
      <td>60.212</td>
      <td>38.041</td>
    </tr>
  </tbody>
</table>
</div></td><td><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-

## ExplainableBoostingRegressor ##

In [40]:
from interpret.glassbox import ExplainableBoostingRegressor


In [41]:
X_train, X_test, y_train, y_test = partitions.split_fold(1)

In [42]:
seed = 42
ebm = ExplainableBoostingRegressor(random_state=seed)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays

ExplainableBoostingRegressor(feature_names=['ENSG00000073921',
                                            'ENSG00000139687',
                                            'ENSG00000135506',
                                            'ENSG00000165995',
                                            'ENSG00000073756',
                                            'ENSG00000097007',
                                            'ENSG00000140519',
                                            'ENSG00000179915',
                                            'ENSG00000068912',
                                            'ENSG00000103479',
                                            'ENSG00000158270',
                                            'ENSG00000158715',
                                            'ENSG00000187955',
                                            'ENSG00000066629',
                                            'ENSG00000157510',
                                            'ENSG000001

In [ ]:
RegressionPerf(ebm.fit(X_train, y_train)

In [47]:
ebm_local = ebm.explain_local(X_test, y_test)
show(ebm_local)

<!-- http://127.0.0.1:7770/140337759185216/ -->

In [45]:
from interpret import show

ebm_global = ebm.explain_global()
show(ebm_global)

<!-- http://127.0.0.1:7770/140337875037584/ -->